In [32]:
import aiohttp
import asyncio
from datetime import datetime, timezone, timedelta
import async_timeout
import pandas

ECOPWR_DAYAHEAD_URL_ACC = "https://acc.ameo.app/api/v2/characteristics/{CURVE}"
ECOPWR_DAYAHEAD_URL = "https://cloud.ameo.app/api/v2/characteristics/{CURVE}"
ECOPWR_HEADERS = {
    "Content-type": "application/json; charset=UTF-8",
    "authorization": "Bearer {TOKEN}",
}


class EcopowerApiClient:
    def __init__(
        self, session: aiohttp.ClientSession, token: str, curve_c, curve_i, test_api
    ) -> None:
        self._token = token
        self._session = session
        self._test_api = test_api
        self._url_c = ECOPWR_DAYAHEAD_URL.format(CURVE=curve_c)
        self._url_i = ECOPWR_DAYAHEAD_URL.format(CURVE=curve_i)
        if self._test_api:
            self._url_c = ECOPWR_DAYAHEAD_URL_ACC.format(CURVE=curve_c)
            self._url_i = ECOPWR_DAYAHEAD_URL_ACC.format(CURVE=curve_i)
        # _LOGGER.debug(f"urls: {self._url_c}, {self.url_i}")

    async def async_get_data(self, url) -> dict:
        async with async_timeout.timeout(8):
            headers = ECOPWR_HEADERS
            headers["authorization"] = f"Bearer {self._token}"
            response = await self._session.get(url, headers=headers)
            response.raise_for_status()
            data = await response.json()
            df = pandas.DataFrame.from_records(data["values"], index="date")
            # df.index = pandas.to_datetime(df.index).tz_convert(tz="Europe/Brussels") 
            # df["value"] = df["value"] * 10**-3  # MWh -> KWh
            # #df.round("H") df = df.drop_duplicates()
            # df = df.groupby(pandas.Grouper(freq='60Min')).mean()
            return df



ecopwr_token = "-4aZXYNX727cqqZhXGU8514RZo6hKf6EMWvqAzXjDEQ"
curve_c = 888
curve_i = 889
ecopower_session = aiohttp.ClientSession()
ecopower_client = EcopowerApiClient(
    ecopower_session, ecopwr_token, curve_c, curve_i, None
)


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fb729c91970>


In [9]:
test =await ecopower_client.async_get_data(ecopower_client._url_c)
test.index = test.index.tz_convert(tz="UTC").floor("H")
test = test.drop_duplicates(ignore_index=False)

In [17]:
path="/home/jan/code_projects/core/config/home-assistant_v2.db"
import pandas
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(path)

In [4]:
def get(self, entity_id: str):
    """Retrieve state of entity_id or None if not found.

    Async friendly.
    """
    val = self._states.get(entity_id.lower())
    if type(val) is dict:
        current_hour = pandas.Timestamp.utcnow().floor("H")
        return val[current_hour]
    else:
        return val


In [30]:
df=pandas.read_sql_query("select * FROM states where entity_id = 'sensor.ecopower_consumption_price_2';", con)
# df.drop_duplicates(subset=['last_updated','context_id'], keep='last')
df.tail(40)

,state_id,entity_id,state,attributes,event_id,last_changed,last_updated,old_state_id,attributes_id,context_id,context_user_id,context_parent_id,origin_idx
0,10,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-16 23:07:10.844005,None,8,01GD48WN3WXT1MW8W96RFWF9VJ,None,None,0
1,41,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-16 23:10:14.558296,None,8,01GD4928GYR9E5NE89Z0MJRGPF,None,None,0
2,76,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-16 23:20:40.158105,None,8,01GD49NBEY1F3FJESDK3S5P8B6,None,None,0
3,162,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-17 22:17:49.946620,None,8,01GD6RF0KTQSYM3HVJM05BZMR8,None,None,0
4,195,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-17 22:23:46.501301,None,8,01GD6RSWT5HMR1MVAKY7Y76BZV,None,None,0
5,228,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-17 22:28:30.068609,None,8,01GD6S2HQMZ6Z89V07JDZ1S1KP,None,None,0
6,261,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-17 22:38:26.726075,None,8,01GD6SMRD6SJDRTS4CFK69K8C4,None,None,0
7,293,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-17 22:45:23.451812,None,8,01GD6T1FBV8GAE0TYNFF3130ME,None,None,0
8,328,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-17 22:53:05.761688,None,8,01GD6TFJV1WW3B1613ED26QHPE,None,None,0
9,360,sensor.ecopower_consumption_price_2,unknown,None,None,None,2022-09-17 22:55:54.039652,None,8,01GD6TMQ5QXERJMXF554PVMEKR,None,None,0


In [27]:
df = pandas.read_sql_query("select shared_attrs FROM state_attributes;", con)
print(df.loc[7,'shared_attrs'])
df

{"unit_of_measurement":"€/kWh","device_class":"monetary","friendly_name":"Ecopower Consumption Price"}


,shared_attrs
0,"{""latitude"":0,""longitude"":0,""radius"":100,""pass..."
1,"{""friendly_name"":""Sun""}"
2,"{""message"":""We have discovered new devices on ..."
3,"{""message"":""Login attempt or request with inva..."
4,"{""editable"":false,""id"":""jan"",""user_id"":""73d9e0..."
5,"{""latitude"":52.3731339,""longitude"":4.8903147,""..."
6,{}
7,"{""unit_of_measurement"":""€/kWh"",""device_class"":..."
8,"{""unit_of_measurement"":""€/kWh"",""device_class"":..."


In [ ]:
"""
type: custom:apexcharts-card
header:
  show: true
  title: ApexCharts-Card
  show_states: true
  colorize_states: true
graph_span: 48h
span:
  start: day
  offset: -0h
now:
  show: true
  label: Now
series:
  - entity: sensor.ecopower_consumption_price
    type: column
    statistics: 
      type: mean
      period: hour
      align: middle
  - entity: sensor.ecopower_injection_price
    type: column
    statistics: 
      type: mean
      period: hour
      align: middle
"""

In [ ]:
insert_df=test.drop(columns='valueStatus')
insert_df.index=insert_df.index.floor("H") 
insert_df= insert_df.reset_index().drop_duplicates(ignore_index=True)#.reset_index()
insert_df

In [ ]:
import homeassistant.util.ulid as ulid_util
import homeassistant.util.dt as dt_util

df = pandas.read_sql_query("select * FROM states;", con)
offset= len(df) + 1

insert_df=test.drop(columns='valueStatus')
insert_df.index=insert_df.index.floor("H") 
insert_df = insert_df.reset_index().drop_duplicates(ignore_index=True)
insert_df['date'] = insert_df['date'].dt.tz_convert(tz=None)
insert_df=insert_df.rename(columns={'date':'last_updated','value':'state'})
insert_df['entity_id']='sensor.ecopower_consumption_price'
insert_df['attributes_id']=8
insert_df['context_id'] = insert_df['last_updated'].apply(lambda x: ulid_util.ulid(dt_util.utc_to_timestamp(x)))
insert_df['origin_idx'] = 0 
insert_df['state_id'] = insert_df.index +  offset
for col in df.columns:
    if col not in insert_df.columns:
        insert_df[col]=None

insert_df

In [ ]:

insert_df.to_sql(
    'states',
    con,
    schema=None,
    if_exists="append",
    index=False,
    # dtype=None,
    # method=None, #change for perf
)


In [ ]:
"""type: custom:apexcharts-card
graph_span: 48h
header:
  title: Electricity Price - Ecopower Injection
  show: true
now:
  show: true
  label: Now
yaxis:
  - decimals: 2
span:
  start: day
  offset: -1d
series:
  - entity: sensor.ecopower_consumption_price
    type: column"""

In [ ]:
pandas.read_sql_query("select * FROM states where entity_id='sensor.ecopower_consumption_price';", con)